In [27]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cvx
import skimage as ski

import skimage.io
import skimage.transform

import scipy.fftpack as spfft # for discrete cosine transform 

# read image
img_og = ski.io.imread('SonOfMan.png')

# convert to grayscale and visualize 
img_og = ski.color.rgb2gray(img_og)

fig, ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].imshow(img_og, cmap = 'gray')
ax[0].set_title("Original image")

print("Original size: ", img_og.shape)

# resize image 
img = ski.transform.rescale( img_og, 0.18, anti_aliasing=False)

print("Rescaled size: ", img.shape)

ax[1].imshow(img, cmap='gray')
ax[1].set_title("Rescaled image")

In [29]:
# constrauct forward and inverse DCT matrices 

# construct DCT matrix
def construct_DCT_Mat( Nx, Ny ):

  # input : Nx number of columns of image 
  #         Ny number of rows of image 

  # output: D DCT matrix mapping image.flatten() to 
  #         DCT(image).flatten()

  Dx = spfft.dct(np.eye(Nx), axis =0, norm='ortho')

  Dy = spfft.dct(np.eye(Ny), axis = 0, norm='ortho')

  D = np.kron(Dy, Dx)

  return D

# construct inverse DCT matrix
def construct_iDCT_Mat( Nx, Ny ):

  # input : Nx number of columns of image 
  #         Ny number of rows of image 

  # output: iD iDCT matrix mapping DCT(image).flatten() to 
  #         image.flatten()

  Dx = spfft.idct(np.eye(Nx), axis =0, norm='ortho')

  Dy = spfft.idct(np.eye(Ny), axis = 0, norm = 'ortho')

  D = np.kron(Dy, Dx)

  return D

In [30]:
img_DCT = construct_DCT_Mat(img.shape[1],img.shape[0])

In [31]:
img_iDCT = construct_iDCT_Mat(img.shape[1],img.shape[0])

In [32]:
F = img.flatten()

In [33]:
DCT_F = np.matmul(img_DCT, F)

fig3, ax3 = plt.subplots()
ax3.set_xlabel('DCT Coefficients')
ax3.set_ylabel('abs( DCT Coefficient Values )')
ax3.plot(abs(DCT_F))

In [35]:
DCT_F_5 = np.copy(DCT_F)
DCT_F_5[abs(DCT_F_5) <= .298] = 0

In [36]:
sum(np.abs(DCT_F_5) > 0)

109

In [37]:
DCT_F_10 = np.copy(DCT_F)
DCT_F_10[abs(DCT_F_10)<= .2098] = 0

In [38]:
sum(np.abs(DCT_F_10)>0)

217

In [39]:
DCT_F_20 = np.copy(DCT_F)
DCT_F_20[abs(DCT_F_20) <= .14195] = 0

In [40]:
sum(np.abs(DCT_F_20)>0)

435

In [41]:
DCT_F_40 = np.copy(DCT_F)
DCT_F_40[abs(DCT_F_40) <= .0833] = 0

In [42]:
sum(np.abs(DCT_F_40)>0)

869

In [53]:
iDCT_5 = np.matmul(img_iDCT,DCT_F_5).reshape((53,41))
iDCT_10 = np.matmul(img_iDCT,DCT_F_10).reshape((53,41))
iDCT_20 = np.matmul(img_iDCT,DCT_F_20).reshape((53,41))
iDCT_40 = np.matmul(img_iDCT,DCT_F_40).reshape((53,41))

images = [iDCT_5,iDCT_10,iDCT_20,iDCT_40]

fig1, ax1 = plt.subplots(1, 4, figsize=(20,5))

labels = [5,10,20,40]

for i in range(4):
        
        ax1[i].imshow(images[i], cmap = 'gray')
        ax1[i].set_title(str(labels[i]) + "% of DCT Coefficients")

## Part 2

In [45]:
vec_F = img.flatten()

In [46]:
N = len(vec_F)

In [47]:
M = [.2 * N, .4 * N, .6 * N]
M = np.around(M).astype(int)
M

array([ 435,  869, 1304])

In [48]:
I = np.eye(N)

In [ ]:
results = np.zeros((3,3,N))

for m in np.arange(len(M)):
    M_m = M[m]
    for j in np.arange(3):
        rand_N = np.random.permutation(N)
        Beta = I[rand_N[0:M_m],:]
        y = np.matmul(Beta,vec_F)
        A = np.matmul(Beta, img_iDCT)
        
        
        x_l1 = cvx.Variable(N)
        objective_l1 = cvx.Minimize( cvx.norm( x_l1, 1 ) )
        constraints_l1 =  [A @ x_l1 == y]
        probl1 = cvx.Problem( objective_l1, constraints_l1)
        
        probl1.solve(verbose=True, solver = 'ECOS', 
                                       reltol=1e-2)
        
        results[m][j][:] = x_l1.value

fig2, ax2 = plt.subplots(3, 3, figsize=(15,15))

for m in range(3):
    for j in range(3):
    
        DCT_F = results[m][j][:]
        
        iDCT_F = np.matmul(img_iDCT,DCT_F).reshape((53,41))
        
        ax2[m][j].imshow(iDCT_F, cmap = 'gray')
        ax2[m][j].set_title('M = ' + str(M[m]))

In [55]:
#fig2.savefig('image_recon.png')
#fig.savefig('rescale.png')
#fig1.savefig('DCT.png')
#fig3.savefig('DCT_recon.png')
#fig4.savefig('random.png')
#fig6.savefig('unknownimage.png')

## random pixel image

ave_results = results.mean(axis = 1)

fig4, ax4 = plt.subplots(3,3, figsize = (15,15))


for m in np.arange(len(M)):
        temp = np.zeros(len(vec_F))
        M_m = M[m]
        rand_N = np.random.permutation(N)

        temp[rand_N[0:M[m]]] = 1

        rand_f = np.ones(2173)
        rand_f[rand_N[0:M[m]]] = vec_F[rand_N[0:M[m]]]
        
        DCT_F_ave = results[m][0][:]
        iDCT_F_ave = np.matmul(img_iDCT,DCT_F_ave).reshape((53,41))

        ax4[0][m].imshow(temp.reshape((53,41)), cmap = 'gray')
        ax4[1][m].imshow(rand_f.reshape((53,41)), cmap = 'gray')
        ax4[2][m].imshow(iDCT_F_ave.reshape((53,41)), cmap = 'gray')
        
        ax4[0][m].set_title('M = ' + str(M[m]))
        ax4[1][m].set_title('M = ' + str(M[m]))
        ax4[2][m].set_title('M = ' + str(M[m]))

fig5, ax5 = plt.subplots()

ax5.imshow(vec_F.reshape((53,41)), cmap = 'gray')

In [50]:
#np.save('rand_pix', results)

## uknown image

unknown = np.load('UnknownImage.npz')

unknown_DCT = construct_DCT_Mat(50,50)
unknown_iDCT = construct_iDCT_Mat(50,50)

B = unknown['B']
y = uknown['y']

A = np.matmul(B, unknown_iDCT)


x_l1 = cvx.Variable(2500)
objective_l1 = cvx.Minimize( cvx.norm( x_l1, 1 ) )
constraints_l1 =  [A @ x_l1 == y]
probl1 = cvx.Problem( objective_l1, constraints_l1)

probl1.solve(verbose=True, solver = 'ECOS', 
                               reltol=1e-2)

fig6, ax6 = plt.subplots(figsize = (10,10))

DCT_unk = x_l1.value

iDCT_unk = np.matmul(unknown_iDCT,DCT_unk).reshape((50,50))

ax6.imshow(iDCT_unk, cmap = 'gray')
ax6.set_title('Unknown Image')